<h1 align="center"> Sensordatenfusion Tutorium 04 </h1>

<h3 align="center"> Messmodel und Updater </h3>


Ziel dieses Tutoriums ist es, $x_{k|k}$ und $P_{k|k}$ zu erhalten. 
Dies wird ermöglicht durch eine (neue) Messung $z_k$. 
Zudem benötigen wir noch die Messmatrix $H$ und ein Rauschen $R_k$.

Das Vorgehen in der Filterung ist nun folgendes:

(1) Berechnung der Innovation $v_{k|k-1}$ und ihrer Kovarianz $S_{k|k-1}$ 

  $v_{k|k-1}$ = $z_k$ - $H_k x_{k|k-1}$
  
  $S_{k|k-1}$ = $H_k P_{k|k-1} H^T_k$ + $R_k$
  
(2) Berechnung der Gewichtsmatrix $W_{k|k-1}$  

  $W_{k|k-1}$ = $P_{k|k-1} H^T_k S_{k|k-1}$


(3) Berechnung $x_{k|k}$ und $P_{k|k}$

  $x_{k|k}$ = $x_{k|k-1} + W_{k|k-1} v_{k|k-1}$ 
  
  $P_{k|k}$ = $P_{k|k-1} - W_{k|k-1} S_{k|k-1} W^T_{k|k-1}$

In [ ]:
import scipy as sp
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.base import Property
from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

In [ ]:
import scipy as sp
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

<h3 align="center"> Zuerst erstellen wir unser Messmodell.</h3>

Fügen Sie die zugehörigen Matrizen in die Funktionen "matrix()" und "covar()" ein
und lassen Sie diese zurückgeben.

Die Funktion "matrix()" gibt die Messmatrix $H$ zurück.

Die Funktion "covar()" gibt die Rauschmatrix $R$ zurück.

In [ ]:
from stonesoup.types.array import CovarianceMatrix

class SDFMessmodell(MeasurementModel, LinearModel, GaussianModel):
    
    def pdf(self):
        pass
    
    @property
    def ndim_meas(self):
        return 2
    
    def matrix(self, **kwargs):
        
        
        return H
    
    def covar(self):
        
        # cov definieren um es als Argument zu übergeben
        
        return CovarianceMatrix(cov)
    
    def rvs(self):
        # sample ziehen aus der Kovarianzmatrix
        noise = multivariate_normal.rvs(sp.zeros(self.ndim_meas), self.covar(), 1)
        return noise.reshape((-1, 1))

In [ ]:
#Messmatrix H
messmodell = SDFMessmodell(4, (0, 2))

<h3 align="center"> Imports für den Updater </h3>

In [ ]:
from functools import lru_cache

from stonesoup.updater.base import Updater
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.prediction import GaussianMeasurementPrediction
from stonesoup.types.update import GaussianStateUpdate

<h3 align="center"> Updater erstellen </h3>

Fügen Sie in die Funktion "update()" alle Berechnungen des Filter-Schrittes ein.

Alle benötigten Komponenten sind in den ersten Zeilen gegeben.

Die "Hypothesis" können Sie vorerst ignorieren.

In [ ]:
class SDFUpdater(Updater):
    def get_measurement_prediction(self, **kwargs):
        pass

    def update(self, hypothesis, measurementmodel, **kwargs):
        
        measurement_mean = hypothesis.measurement.state_vector
        measurement_matrix = measurementmodel.matrix()  # H
        measurement_noise_covar = measurementmodel.covar()  # R
        prediction_covar = hypothesis.prediction.covar  # P
        prior = hypothesis.prediction.mean
        
        
        hypothesis = SingleHypothesis(hypothesis.prediction,
                                      hypothesis.measurement,
                                      GaussianMeasurementPrediction(
                                          messprediction, S,
                                          hypothesis.prediction.timestamp)
                                      )

        return GaussianStateUpdate(x_post,
                                   P_post,
                                   hypothesis,
                                   hypothesis.measurement.timestamp)